## Agent models and decision processes: key concepts with examples - reinforcement and imitation learning

In the field of artificial intelligence, when one wishes to create a system that observes the environment and performs some kind of action in response, one generally thinks of this as a **decision process**. This is of particular interest to researchers seeking to employ machine learning methods in robotics. When reading discourse surrounding this broad topic, it is quite common to come across terms such as "agents", "policies", "states", "actions", "reinforcement learning", "reward functions" and so forth. This notebook is intended to serve as a gentle introduction to these concepts and the more common theoretical frameworks associated with them. It also includes some simple worked examples of learning a policy for balancing a pole on a cart using reinforcement and imitation learning

### Overview

Broadly and informally, a **decision process** is a process where an **agent** takes an **observation** of the **state** of its **environment** and comes up with an **action** it should take using a **policy**. Now, that's a lot of big words all at once, so in more detail about each:

- **environment** -- a real or simulated "world" where objects exist in some configuration (its **state**). For the purposes of decision processes, it is usually assumed that this world has time which progresses in discrete steps;
- **state** -- a vector fully describing the configuration of everything that exists in the environment. This changes with time, and since time progresses in discrete steps, any environment can be described as a sequence of its states as they evolve over time $ (s_1, s_{2}, ... ) $. The state belongs to a set of all possible states $ s \in S $. The variables that make up this **state** vector can be discrete or continuous;
- **observation** -- a vector that includes some subset of the **state** variables. This is necessary because in most cases, the complete **state** of the system is impractical or impossible to observe. If the **observation** gives us enough information to fully determine the future **state** of everything we care about, it is generally treated the same as if it were the complete **state**;
- **agent** -- an entity, such as a robot controller, animal, human or AI subroutine managing a character in a video game, that exists in some sort of **environment**. What sets an agent apart from just any old object is that it can **observe** the **state** of its **environment** and possesses the means to alter it in some way (such as a means of locomotion to alter its own position, or actuators to move other things);
- **action** -- something the **agent** can do to influence the **state** of the environment. The **action** belongs to a set of all possible actions $ a \in A $ This is typically paired with a **transition function** which describes how the future state of the environment is altered when a given **action** is taken at a given **state**. The **transition function** can be deterministic $ T(s_t, a) = s_{t+1}$ or probabilistic $ T(s_t, s_{t+1}, a) = P(s_{t+1} \vert s_t, a) $. The variables that make up the **action** vector can be discrete or continuous, but in most classic reinforcement and imitation learning contexts we only deal with discrete actions;
- **policy** -- a function which dictates how the agent will respond to any given state in its environment. 

Now that we're familiar with the terms, we can turn to defining a specific **decision process**. There are many ways one could theoretically go about doing this, but in practice almost every paper related to reinforcement and imitation learning (until some very recent developments, see below) starts by laying down the definition of a **Markov Decision Process**, or MDP for short. There is some variability in how people choose to state it, but usually it will be something like

$$ MDP = (S, A, R, T) $$

where $ S $ represents the set of all possible states, $ A $ is the set of all possible actions, $ R: S \rightarrow \mathbb{R} $ is the **reward** function which tells us how well we did by ending up in this state and $ T: S \times A \rightarrow S \text{ or } P(S) $ is the (deterministic or probabilistic) transition function. There's quite a bit of wiggle room over the specifics of stating this (introducing objects such as $ \Gamma : S \rightarrow A $, a function that tells us what actions are available at which states) but the main takeaway here is that **the future state of the system depends only on the current state and the action taken by the agent**. This means there is no hysteresis -- we don't care about anything that happened before the current state or how we got here. Thus most of the theory dealing with this concept also assumes that any given agent will define the **policy** as $ \pi : S \rightarrow A \text{ or } P(A) $. 

The primary question the fields of **reinforcement and imitation learning** seek to answer is thus: *given a Markov Decision Process, what is the most effective way of discovering a policy function?* Both are discussed in more detail below, but the long and short of it is:

- In **reinforcement learning** we give our agent a random starting policy (typically in the form of some model template, an object which can approximate a wide range of possible functions -- such as a neural network) and let it loose upon the environment. This way we collect rewards which tell us how well we did. Using the rewards, we update the policy, then try again. Repeat until a useful agent pops out. The biggest advantage of this approach is that we don't have to know anything about how to actually achieve the goal, nor do we need a system that can. In theory, we only need to be able to define the reward function. In practice, however, there are several important drawbacks. First of all, this method is computationally extremely expensive, since the only way to learn is through interaction with the (typically simulated) environment. Secondly, many problems have extremely sparse rewards (meaning we can go a long time before seeing how well we did), which results in astronomically huge search spaces that agents have to explore before getting any useful feedback. Thus, the approach of
- **imtation learning** takes example state-action pairs produced by an **expert** and trains a policy that can reproduce these accurately. Obviously, this only works if we have an expert (such as a human demonstrator) to begin with. But provided we do, this lets us skip the incredibly expensive exploration phase of the process and skip straight to learning. Furthermore, a policy obtained from a not very good expert can then be improved through reinforcement learning, and lots of research in the field is about precisely this -- using imitation learning to "kickstart" a reinforcement learning process.

So far, we've only discussed cases when the full state of the system is known and it follows the Markov Decision Process formalism. Things get somewhat murkier once we start dealing with incomplete observations -- it is possible that some hidden variables are not apparent from any given observation but can in fact be inferred from longer state histories. This is particularly often the case when dealing with agents which will have to operate in real rather than simulated environments -- such as robot controllers. Therefore the most recent state-of-the-art research, particularly regarding imitation learning, has been quite heavily slanted towards using sequence prediction models (such as transformer neural networks) which also keep track of past states, moving us away from the world of MDPs. However, in many cases methods grounded in the MDP are perfectly sufficient and those are the only ones used in examples below.

### Reinforcement learning

In [ ]:
from QMatrix import QMatrix, INNAME
from utils import visualize_model

qmat = QMatrix.load(f"{INNAME}.pickle")
visualize_model(qmat, notebook=True)

### Imitation learning